In [28]:
import tensorflow as tf

In [29]:
# Load (and cache) standard MNIST dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Extract the pixel counts.
input_shape = x_train.shape[1] * x_train.shape[2]

# Reshape and rescale the train and test data.
x_train = x_train.reshape(x_train.shape[0], input_shape) / 255.
x_test = x_test.reshape(x_test.shape[0], input_shape) / 255.

# Extract the number of classes.
n_classes = len(set(y_train))

# Convert the class numbers to 1-hot categorical values.
y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_test = tf.keras.utils.to_categorical(y_test, n_classes)

In [30]:
# Build a single softmax categorical layer.
model = tf.keras.Sequential(
    tf.keras.layers.Dense(
        units=n_classes,
        input_shape=(input_shape,),
        activation='softmax',
    )
)

# Compile using 'Adam'
model.compile(
    optimizer='adam',
    # Needed for the categorical softmax layer.
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Print the model summary.
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=50,
    verbose=1,
    validation_split=0.2,
)

# Evaluate the model with the test data.
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

Epoch 1/50
375/375 [==============================] - 1s 2ms/step - loss: 0.7497 - accuracy: 0.8179 - val_loss: 0.4108 - val_accuracy: 0.8970
Epoch 2/50
375/375 [==============================] - 1s 1ms/step - loss: 0.3908 - accuracy: 0.8968 - val_loss: 0.3378 - val_accuracy: 0.9099